In [2]:
include("voiConsAuc.jl")

evsi_plot (generic function with 3 methods)

In [3]:
function es(μ = [0, 0], σ = [1, 1])
    function g(n)
        f(p) = evsi_sim(μ, σ, [n * p, n * (1 - p)], nsims = 100000)
        vcat(map(f, 1:-.01:0)...)
    end
    hcat(map(g, exp(collect(linspace(log(1), log(50), 101))))...)
end

es (generic function with 3 methods)

In [4]:
x = es([0, 0], [1, 1])

101x101 Array{Float64,2}:
 0.278762  0.285481  0.283648  0.287578  …  0.393385  0.394209  0.397694
 0.285904  0.285447  0.288884  0.292556     0.451202  0.456243  0.459097
 0.289782  0.287031  0.294505  0.298489     0.483687  0.483495  0.487337
 0.279098  0.290551  0.291719  0.289424     0.497177  0.501509  0.503215
 0.283744  0.292489  0.298435  0.301355     0.511045  0.509361  0.515337
 0.293904  0.291569  0.301181  0.302129  …  0.514212  0.520872  0.517178
 0.294383  0.293472  0.301306  0.302444     0.52427   0.524195  0.52146 
 0.295277  0.298837  0.302113  0.306311     0.528252  0.528976  0.526048
 0.298799  0.300173  0.302228  0.308829     0.529678  0.533603  0.533201
 0.298042  0.300867  0.311922  0.304272     0.532201  0.534054  0.536091
 0.295844  0.305417  0.307951  0.312276  …  0.538258  0.534027  0.538195
 0.299735  0.304665  0.304188  0.307608     0.538784  0.540261  0.538102
 0.299889  0.301376  0.315202  0.308104     0.534745  0.539705  0.5406  
 ⋮                       

In [17]:
R"
    library(raster)
    library(mgcv)
    r = $x
    r = 
      data.frame(
        x = rep(seq_len(ncol(r)), each = nrow(r)),
        y = rep(seq_len(nrow(r)), times = ncol(r)),
        z = c(r)
      )
    r = gam(z ~ te(x, y), data = r)
    r = matrix(fitted(r), ncol = 101)
    #r = t(t(r) / apply(r, 2, max))
    r = raster(r)
    png('plot.png')
    plot(r, asp = NA, las = 1, xaxt = 'n')
    contour(r, add = TRUE)
    axis(1, at = 0:5 / 5, label = round(exp(seq(log(1), log(50), length.out = 6))))
    dev.off()
"

RCall.RObject{RCall.IntSxp}
null device 
          1 
